In [21]:
from openai import OpenAI
import gradio as gr
import json


ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}
MODEL = "llama3.1:latest"
def get_ticket_price(destination_city):
    print(f"Tool called for city {destination_city}")
    price = ticket_prices.get(destination_city.lower(), "Unknown ticket price")
    return f"The price of a ticket to {destination_city} is {price}"

tools = [{"type": "function", "function": get_ticket_price}]

system_message = """
You are assitant for doing multiple calculations.
"""

get_ticket_price = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

def chatfn(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}] 
    response = ollama.chat.completions.create(model=MODEL, messages=messages)
    print(f"Content received from LLM {response.choices[0].message.content}")
    #return response.choices[0].message.content
    return response.choices[0].message.content

 

def chatHTMLfn(message="hello", history="dsafdsf"):
    # Extract numbers
    nums = [int(n) for n in message.replace(",", " ").split() if n.isdigit()]
    if not nums:
        nums = [10, 20, 30]

    categories = [f"Point {i+1}" for i in range(len(nums))]

    config = {
        "chart": {"type": "line"},
        "title": {"text": "Generated Line Chart"},
        "xAxis": {"categories": [10,20,30]},
        "yAxis": {"title": {"text": "Values"}},
        "series": [{"name": "Series 1", "data": [1,2,3]}]
    }

    html = f"""
    <div id="hc_chart" style="height: 400px;"></div>
    <script src="https://code.highcharts.com/highcharts.js"></script>
    <script>
        Highcharts.chart('hc_chart', ${config})
    </script>
    """
    print(f"The Formatted data is {getHTML()}")
    #return gr.HTML(value= f"""<div id="hc_chart" style="height: 400px;">Hello</div>""")
    with gr.blocks(head = """
        <script src="https://code.highcharts.com/highcharts.js"></script>
         <script>
        Highcharts.chart('hc_chart', ${config})
     </script>""") as demo:
        gr.HTML(value= f"""<div id="hc_chart" style="height: 400px;"></div>""")
    return demo    

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == "get_ticket_price":
        print(f"Tool called for city {tool_call}")
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price_details = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": price_details,
            "tool_call_id": tool_call.idrty56
        }
    return response

OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')
#chatHTMLfn()
with gr.blocks(head = """
    <script src="https://code.highcharts.com/highcharts.js"></script>
    <script>
        Highcharts.chart('hc_chart', ${config})
     </script>""") as hc:
    gr.ChatInterface(fn=chatHTMLfn, type="messages").launch()





TypeError: 'module' object is not callable